# 03 - ré-Processamento e Construção do Pipeline (scikit-learn)

<a href="../README.md" title="Voltar para a página principal">
🏠 Voltar para Home
</a>

## Visão Geral

Este notebook implementa o *pré-processamento completo* do dataset para modelos de Machine Learning, seguindo boas práticas recomendadas pela biblioteca scikit-learn.

## O pipeline final incluirá:

1. Separação entre features numéricas e categóricas
2. Imputação de valores ausentes
3. Codificação de variáveis categóricas (One-Hot Encoding)
4. Padronização de variáveis numéricas
5. Montagem do ColumnTransformer
6. Geração do conjunto final X_prepared e y

Este notebook prepara os dados para os próximos passos:

- 04_modelos_classicos.ipynb
- 05_modelos_avancados.ipynb

# Importações

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import (
    StandardScaler,
    OneHotEncoder
)
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

import warnings
warnings.filterwarnings("ignore")

# Trabalho com html
from IPython.display import display, HTML

# 1. Carregamento e Visualização Preliminar

***Descrição:*** Utilizando o dataset de acompanhamento operacional dos pedidos tratados.
> **Arquivo e:** database/processed/acompanhamento_operacional_FE.csv

In [8]:
file_path = "../database/processed/acompanhamento_operacional_FE.csv"

df = pd.read_csv(file_path)

print("Dataset carregado com sucesso!")
print("Formato:", df.shape)
df.head()

Dataset carregado com sucesso!
Formato: (488398, 29)


,sigla_cliente,tipo_veiculo,qtde_itens,volume,peso,m3,uf,fl_base,representante,flag_entrega_agendada,...,horas_conferencia,horas_emissao,horas_analise_producao,horas_minuta,horas_exped_minuta,hora_analise_transporte,lead_time_total_horas,complexidade_operacional,pedido_grande_flag,processo_longo_flag
0,NTL,TRUCK 70 M3,1,1,144.30,2.10,MS,0,NÃO DEFINIDO,Não,...,0.0,0.0,22.0,0.0,16.0,161.0,229.0,1,0,0
1,NTL,TRUCK 70 M3,1,1,130.30,2.33,MS,0,NÃO DEFINIDO,Não,...,0.0,0.0,22.0,0.0,16.0,165.0,234.0,1,0,0
2,NTL,TRUCK 70 M3,1,1,97.00,2.22,SC,0,SC CARGO TRANSPORTES LTDA,Não,...,0.0,0.0,0.0,0.0,16.0,40.0,249.0,1,0,0
3,MMM,TRUCK 75 M3,7,43,182.79,1.08,CE,0,VELOMAX BRASIL TRANSPORTES LTD,Não,...,1.0,0.0,11.0,99.0,8.0,93.0,266.0,7,1,0
4,MMM,TRUCK 70 M3,4,9,25.50,0.07,CE,0,MFM TRANSPORTES,Sim,...,1.0,0.0,12.0,172.0,16.0,285.0,542.0,4,1,1


# 2. Identificar Variáveis Numércias e Categóricas

In [10]:
TARGET = "fl_atraso_cli"

# Remover target para análise inicial
cols = df.columns.tolist()

num_cols = df.select_dtypes(include=['int64','float64']).columns.tolist()
num_cols = [c for c in num_cols if c != TARGET]

cat_cols = df.select_dtypes(include=['object']).columns.tolist()

# Cria o DataFrame-resumo
df_summary = pd.DataFrame({
    'tipo': ['numerico', 'categorico'],
    'qtde': [len(num_cols), len(cat_cols)],
    'features': [num_cols, cat_cols]
})

html = f"""
<div style="display: flex; gap: 50px; justify-content: left;">

    <div>
        <h3 style="text-align:center;">Summary</h3>
        {df_summary.to_html(index=True)}
    </div>

</div>
"""

display(HTML(html))

,tipo,qtde,features
0,numerico,22,"[qtde_itens, volume, peso, m3, fl_base, qtde_ocams, peso_cubado_rodoviario, horas_pre_conferencia, horas_distribuicao_cotas, horas_planejamento, horas_divisao_ocam, horas_coleta, horas_conferencia, horas_emissao, horas_analise_producao, horas_minuta, horas_exped_minuta, hora_analise_transporte, lead_time_total_horas, complexidade_operacional, pedido_grande_flag, processo_longo_flag]"
1,categorico,6,"[sigla_cliente, tipo_veiculo, uf, representante, flag_entrega_agendada, modalidade]"
